# Funciones para los Endpoints proporcionados
En esta sección, nuestro enfoque estará en utilizar los archivos generados en el notebook de Consultas, los cuales serán consumidos por la API. El objetivo principal es proporcionar respuestas a los Endpoints y a los Modelos de Machine Learning de manera rápida y eficiente durante la ejecución de la API. Para lograr esto, desarrollaremos funciones específicas que permitirán acceder y procesar los datos necesarios de manera óptima, asegurando un rendimiento adecuado de la API.

In [1]:
import pandas as pd
import os

In [2]:
# Cargar los datos limpios
df_UserItems = pd.read_parquet("C:\\Users\\Gary Alexander Bean\\Desktop\\Proyecto-Individual-1-MLOps\\Datasets\\ArchivosPARQUET\\user_items_limpio.parquet")
df_UserReviews = pd.read_parquet("C:\\Users\\Gary Alexander Bean\\Desktop\\Proyecto-Individual-1-MLOps\\Datasets\\ArchivosPARQUET\\user_reviews_limpio.parquet")
df_SteamGames = pd.read_parquet("C:\\Users\\Gary Alexander Bean\\Desktop\\Proyecto-Individual-1-MLOps\\Datasets\\ArchivosPARQUET\\steam_games_limpio.parquet")

# Endpoint 1

- **def developer( desarrollador : str ):** Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de retorno: 

Año	   Cantidad de Items  	Contenido Free
2023	     50	                27%
2022	     45	                25%
xxxx	     xx	                xx%

In [3]:
# Funcion para el endpoint 'developer'
def developer(desarrollador: str):
    # Filtrar el DataFrame de juegos por la desarrolladora especifica
    developer_df = df_SteamGames[df_SteamGames['developer']== desarrollador]

    # Contar la cantidad de juegos y calcular el porcentaje de contenido Free por año
    developer_data = developer_df.groupby('year').agg({'item_id': 'count', 'Free to Play': 'mean'}).reset_index()
    developer_data.rename(columns={'item_id': 'Cantidad de Items', 'Free to play': 'Contenido Free'}, inplace=True)
    developer_data['Contenido Free']= developer_data['Contenido Free'].apply(lambda x: f"{x: .0%}")

    # Convertir el resultado a formato JSON
    result = developer_data.to_dict(orient='records')
    return result


# Endpoint 2
- **def userdata( User_id : str ):** Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.
Ejemplo de retorno: {"Usuario X" : us213ndjss09sdf, "Dinero gastado": 200 USD, "% de recomendación": 20%, "cantidad de items": 5}

In [4]:
# Funcion para el endpoint 'userdata'
def userdata(User_id: str):
    # Filtrar el DataFrame de reseñas por el usuario especifico
    user_df = df_UserReviews[df_UserReviews['user_id'] == User_id]

    # Calcular el dinero gastado, el porcentaje de recomendacion y la cantidad de items
    money_spent = user_df['total_spend'].sum()
    recomend_percentage = user_df['recommend'].mean()
    items_count = len(user_df)

    # Convertir el resultado a formato JSON
    result = {
        "Usuario": User_id,
        "Dinero gastado": f"{money_spent:.2f} USD",
        "% de recomendacion": f"{recommend_percentage: .0%}",
        "Cantidad de items": items_count
    }
    return result

# Endpoint 3
- **def UserForGenre( genero : str ):** Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año de lanzamiento.
Ejemplo de retorno: {"Usuario con más horas jugadas para Género X" : us213ndjss09sdf, "Horas jugadas":[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [5]:
# Funcion para el endpoint 'UserForGenre'
def UserForGenre(genero: str):
    # Filtrar el DataFrame de reseñas por el genero especifico
    genre_df = df_UserItems[df_UserItems[genero] == 1]

    # Agrupar por usuario y año, sumar las horas jugadas y encontrar el usuario con mas jugadas
    total_hours_by_user_and_year = genre_df.groupby(['user_id', 'year'])['playtime_forever'].sum()
    max_user = total_hours_by_user_and_year.groupby('user_id').sum().idxmax()

    # Obtener la lista de acumulacion de horas jugadas por año para el usuario con mas horas jugadas
    max_user_hours_by_year = total_hours_by_user_and_year.loc[max_user].reset_index()
    max_user_hours_list = [{"Año": int(row['year']), "Horas": row['playtime_forever']} for _, row in max_user_hours_by_year.interrows()]

    # Convertir el resultado a formato JSON
    result= {
        "Usuario con mas horas jugadas para Genero {}".format(genero): max_user,
        "Horas jugadas": max_user_hours_list
    }
    return result

# Endpoint 4
- **def best_developer_year( año : int ):** Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [6]:
# Funcion para el endpoint 'best_developer_year
def best_developer_year(año: int):
    # Filtrar el DataFrame de reseñas por el año específico y por recomendaciones positivas
    positive_reviews_df = df_UserReviews[(df_UserReviews['year'] == año) & (df_UserReviews['recommend'] == True) & (df_UserReviews['sentiment_analysis'].isin([1, 2]))]
    
    # Hacer un join entre las reseñas positivas y los juegos para obtener el nombre del juego y el desarrollador
    merged_reviews = pd.merge(positive_reviews_df, df_SteamGames[['item_id', 'name', 'developer']], on='item_id', how='left')
    
    # Contar la cantidad de juegos recomendados por desarrollador
    developer_count = merged_reviews['developer'].value_counts()
    
    # Seleccionar los top 3 de desarrolladores con más juegos recomendados
    top_3_developers = developer_count.head(3)
    
    # Convertir el resultado a formato JSON
    result = [{"Puesto {}".format(i+1): developer} for i, (developer, _) in enumerate(top_3_developers.items())]
    return result

# Endpoint 5
- **def developer_reviews_analysis( desarrolladora : str ):** Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.
Ejemplo de retorno: {'Valve' : [Negative = 182, Positive = 278]}

In [7]:
# Función para el endpoint 'developer_reviews_analysis'
def developer_reviews_analysis(desarrolladora: str):
    # Filtrar el DataFrame de reseñas por la desarrolladora específica
    reviews_by_developer = df_UserReviews[df_UserReviews['developer'] == desarrolladora]
    
    # Calcular el análisis de sentimiento
    sentiment_counts = reviews_by_developer['sentiment_analysis'].value_counts()
    
    # Convertir el resultado a formato JSON
    result = {desarrolladora: {
        'Negative': sentiment_counts.get(0, 0),
        'Neutral': sentiment_counts.get(1, 0),
        'Positive': sentiment_counts.get(2, 0)
    }}
    return result